# Classifying Celestial Objects
Based on information from the dataset neo_v2 from https://www.kaggle.com/datasets/sameepvani/nasa-nearest-earth-objects, this neural network classifies whether asteroids may pose a danger to earth based on a variety of features. 

### Imports

In [1]:
import tensorflow as tf 
from tensorflow import keras 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import io
import os
import tempfile

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

relative_velocity_flag = False
miss_distance_flag = False

### Data preprocessing
The neo_v2 csv file is imported from the REHS github repo. Then the program splits the dataset 80/20 into training data and testing data respectively

In [2]:
csv_link = 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/neo_v2.csv'
csv_file = requests.get(csv_link).content

In [3]:
dataset = pd.read_csv(io.StringIO(csv_file.decode('utf-8')))

In [4]:
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,False,20.09,True


convert boolean values to floats

In [5]:
print(dataset.dtypes.tolist())
print(type(dataset.dtypes.tolist()[0]))

change_to_float = {column:float for column, dtype in zip(dataset.columns.tolist(), dataset.dtypes.tolist()) if dtype == np.bool_}
dataset = dataset.astype(change_to_float)
dataset.head()

[dtype('int64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('bool'), dtype('float64'), dtype('bool')]
<class 'numpy.dtype[int64]'>


,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,0.0,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,0.0,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,0.0,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,0.0,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,0.0,20.09,1.0


get rid of orbiting body and sentry_object

In [6]:
try:
    dataset.drop('sentry_object', axis=1, inplace=True)
except (AttributeError, KeyError):
    print("already removed")

try:
    dataset.drop('orbiting_body', axis=1, inplace=True)
except (AttributeError, KeyError):
    print("already removed")

In [7]:
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,20.09,1.0


what percent of our data is positive?

In [8]:
neg, pos = np.bincount(dataset['hazardous'])
total = neg + pos
print(f"Percent positive is {(pos / neg) * 100} %")

Percent positive is 10.781013707985755 %


In [9]:
dataset_raw = dataset # to look at later

In [10]:
def scale_down(dataset=None, column=None):
    initial_range = np.amax(dataset[column]) - np.amin(dataset[column])
    eps = 0.001
    dataset[column] = np.log(dataset[column] + eps)
    end_range = np.amax(dataset[column]) - np.amin(dataset[column])

    return initial_range, end_range

In [11]:
# try to reduce the range of the miss distance
if not relative_velocity_flag:
    initial_range, end_range = scale_down(column='miss_distance', dataset=dataset)
    relative_velocity_flag = True

else:
    print("scaling already complete")

In [12]:
# try to reduce the range of the relative velocity
if not miss_distance_flag:
    initial_range, end_range = scale_down(column='relative_velocity', dataset=dataset)
    miss_distance_flag = True
else:
    print("scaling already complete")

In [13]:
scaler = StandardScaler()

In [14]:
train_data, test_data = train_test_split(dataset.drop(['id', 'name'], axis=1), test_size=0.2)
train_data, validation_data = train_test_split(train_data, test_size=0.2)

train_labels, test_labels, validation_labels = (np.array(train_data.pop('hazardous')),
                                                np.array(test_data.pop('hazardous')), 
                                                np.array(validation_data.pop('hazardous')))
bool_train_labels = train_labels != 0
train_features, test_features, validation_features = (np.clip(scaler.fit_transform(np.array(train_data)), 0, 60),
                                                      np.clip(scaler.fit_transform(np.array(test_data)), 0, 60), 
                                                      np.clip(scaler.fit_transform(np.array(validation_data)), 0, 60))

In [15]:
model_metrics = [keras.metrics.BinaryAccuracy(),
                       keras.metrics.Precision(),
                       keras.metrics.Recall(),
                       keras.metrics.TruePositives(),
                       keras.metrics.TrueNegatives(),
                       keras.metrics.FalseNegatives(),
                       keras.metrics.FalsePositives(),
                       keras.metrics.AUC(name='auc'),
                       keras.metrics.AUC(name='prc', curve='PR')
                       ]

In [112]:
def make_model(metrics=model_metrics, output_bias=None):
    if output_bias:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = tf.keras.Sequential([
        keras.layers.Dense(5, activation='relu', input_shape=(train_features.shape[-1],), 
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4),
                            activity_regularizer=keras.regularizers.L2(1e-5)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4, activation='relu', input_shape=(train_features.shape[-1],),
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4),
                            activity_regularizer=keras.regularizers.L2(1e-5)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(3, activation='relu', input_shape=(train_features.shape[-1],),
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4),
                            bias_regularizer=keras.regularizers.L2(1e-4),
                            activity_regularizer=keras.regularizers.L2(1e-5)),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    
    return model

In [110]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True
)

In [113]:
model = make_model()

In [114]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 5)                 30        
                                                                 
 dropout_3 (Dropout)         (None, 5)                 0         
                                                                 
 dense_5 (Dense)             (None, 4)                 24        
                                                                 
 dropout_4 (Dropout)         (None, 4)                 0         
                                                                 
 dense_6 (Dense)             (None, 3)                 15        
                                                                 
 dropout_5 (Dropout)         (None, 3)                 0         
                                                                 
 dense_7 (Dense)             (None, 1)                

getting the initial bias

In [115]:
# unbiased
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=1)
print(f'Loss: {results[0]}')

29/29 [==============================] - 2s 4ms/step - loss: 1.5255 - binary_accuracy: 0.2620 - precision: 0.1121 - recall: 0.9592 - true_positives: 7070.0000 - true_negatives: 12923.0000 - false_negatives: 301.0000 - false_positives: 56008.0000 - auc: 0.4751 - prc: 0.0887
Loss: 1.5254980325698853


initial_bias = ln(pos/neg)
where positive is the number of features classified positive, and neg is the number of features classified as negative

In [116]:
initial_bias = np.log([pos/neg])
initial_bias

array([-2.22738359])

In [117]:
model = make_model(output_bias=initial_bias)

In [118]:
model.predict(train_features[:10])

1/1 [==============================] - 0s 157ms/step


array([[0.10336942],
       [0.10361035],
       [0.09796359],
       [0.09731824],
       [0.10566195],
       [0.09731824],
       [0.09856623],
       [0.10730299],
       [0.09782594],
       [0.09731824]], dtype=float32)

In [119]:
# biased
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=1)
print(f'Loss: {results[0]}')

29/29 [==============================] - 2s 3ms/step - loss: 0.3195 - binary_accuracy: 0.5392 - precision: 0.1121 - recall: 0.5436 - true_positives: 7070.0000 - true_negatives: 65418.0000 - false_negatives: 5936.0000 - false_positives: 56012.0000 - auc: 0.4909 - prc: 0.0922
Loss: 0.3195318281650543


save a benchmark

In [120]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [121]:
model = make_model()
model.load_weights(initial_weights)

baseline = model.fit(train_features, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping], validation_data=(validation_features, validation_labels))

Epoch 1/100
29/29 [==============================] - 6s 67ms/step - loss: 0.3375 - binary_accuracy: 0.6467 - precision: 0.1115 - recall: 0.3803 - true_positives: 7090.0000 - true_negatives: 117436.0000 - false_negatives: 11551.0000 - false_positives: 56493.0000 - auc: 0.4913 - prc: 0.0928 - val_loss: 0.3284 - val_binary_accuracy: 0.8989 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 13065.0000 - val_false_negatives: 1469.0000 - val_false_positives: 0.0000e+00 - val_auc: 0.5340 - val_prc: 0.1267
Epoch 2/100
29/29 [==============================] - 0s 7ms/step - loss: 0.3264 - binary_accuracy: 0.9007 - precision: 0.0205 - recall: 5.3239e-04 - true_positives: 3.0000 - true_negatives: 52356.0000 - false_negatives: 5632.0000 - false_positives: 143.0000 - auc: 0.4880 - prc: 0.0912 - val_loss: 0.3288 - val_binary_accuracy: 0.8989 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negativ

class weights

In [122]:
weight_for_0 = (0.94 / neg) * (total / 2.0)
weight_for_1 = (0.75 / pos) * (total / 2.0)

class_weights = {0: weight_for_0, 1: weight_for_1}

print(f'Non-Hazardous Weight: {weight_for_0}\nHazardous Weight: {weight_for_1}')

Non-Hazardous Weight: 0.520670764427533
Hazardous Weight: 3.853337104072398


In [123]:
tf.keras.backend.clear_session()

In [106]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

In [67]:
# weighted_history = weighted_model.fit(train_features, 
#                                     train_labels, 
#                                     batch_size=BATCH_SIZE, 
#                                     epochs=EPOCHS, 
#                                     callbacks=[early_stopping], 
#                                     validation_data=(validation_features, validation_labels), 
#                                     class_weight=class_weights)

try training it with just the weights

In [124]:
weighted_model.fit(train_features, 
                    train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    #callbacks=[early_stopping], 
                    validation_data=(validation_features, validation_labels), 
                    class_weight=class_weights)

Epoch 1/100
29/29 [==============================] - 1s 25ms/step - loss: 0.5215 - binary_accuracy: 0.7919 - precision: 0.2268 - recall: 0.4760 - true_positives: 2682.0000 - true_negatives: 43357.0000 - false_negatives: 2953.0000 - false_positives: 9142.0000 - auc: 0.7378 - prc: 0.2137 - val_loss: 0.4848 - val_binary_accuracy: 0.7896 - val_precision: 0.3036 - val_recall: 0.8359 - val_true_positives: 1228.0000 - val_true_negatives: 10248.0000 - val_false_negatives: 241.0000 - val_false_positives: 2817.0000 - val_auc: 0.8805 - val_prc: 0.3793
Epoch 2/100
29/29 [==============================] - 0s 7ms/step - loss: 0.5241 - binary_accuracy: 0.7922 - precision: 0.2233 - recall: 0.4614 - true_positives: 2600.0000 - true_negatives: 43454.0000 - false_negatives: 3035.0000 - false_positives: 9045.0000 - auc: 0.7332 - prc: 0.2125 - val_loss: 0.4841 - val_binary_accuracy: 0.8016 - val_precision: 0.3095 - val_recall: 0.7822 - val_true_positives: 1149.0000 - val_true_negatives: 10501.0000 - val_fa

In [125]:
weighted_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

9/9 [==============================] - 0s 3ms/step - loss: 0.4855 - binary_accuracy: 0.7872 - precision: 0.2889 - recall: 0.8393 - true_positives: 1457.0000 - true_negatives: 12845.0000 - false_negatives: 279.0000 - false_positives: 3587.0000 - auc: 0.8622 - prc: 0.3623


[0.485504150390625,
 0.7872082591056824,
 0.2888580560684204,
 0.8392857313156128,
 1457.0,
 12845.0,
 279.0,
 3587.0,
 0.8622347116470337,
 0.3622530698776245]

In [138]:
results = weighted_model.predict(test_features)
results[results<0.5]=0
results[results>0.5]=1

568/568 [==============================] - 1s 2ms/step


TypeError: array() missing required argument 'object' (pos 0)

In [148]:
results.shape
results = results.reshape(18168,)
comparison = np.stack([results, test_labels, test_])

In [154]:
comparison_tuples = list(zip(comparison[0], comparison[1]))

In [152]:
def check_correctness(comparison):
    for predicted_label, actual_label in comparison:
        if predicted_label == actual_label:
            print(True)
            continue
        print(False)

In [155]:
check_correctness(comparison_tuples)

True
False
True
False
True
True
False
True
True
False
True
True
True
False
True
False
True
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
False
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
False
True
True
False
True
True
False
True
True
True
False
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
False
True
False
True
True
False
False
True
True
True
True
True
True
False
False
True
True
True
False
True
True
True
False
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
True
False
True
False
True
True
False
True
True
True


Oversampling

In [32]:
# pos_features, neg_features = train_features[bool_train_labels], train_features[~bool_train_labels]

# pos_labels, neg_labels = train_labels[bool_train_labels], train_labels[~bool_train_labels]

In [33]:
# BUFFER_SIZE = 100000

# def make_ds(features, labels):
#     ds = tf.data.Dataset.from_tensor_slices((features, labels))
#     ds = ds.shuffle(BUFFER_SIZE).repeat()
#     return ds

In [34]:
# pos_ds, neg_ds = make_ds(pos_features, pos_labels), make_ds(neg_features, neg_labels)

In [35]:
# resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
# resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

# resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)

train with oversampling

In [36]:
# resampled_model = make_model()
# resampled_model.load_weights(initial_weights)

# # reset bias to 0, the set is now balanced
# output_layer = resampled_model.layers[-1]
# output_layer.bias.assign([0])

# validation_data = tf.data.Dataset.from_tensor_slices((validation_data, validation_labels)).cache()
# validation_data = validation_data.batch(BATCH_SIZE).prefetch(2)

In [37]:
# resampled_history = resampled_model.fit(resampled_ds,
#                                         epochs=EPOCHS,
#                                         steps_per_epoch=resampled_steps_per_epoch,
#                                         callbacks=[early_stopping],
#                                         validation_data=validation_data)

make the resample model

In [38]:
# resampled_model.fit(resampled_ds,
#                     epochs=EPOCHS,
#                     steps_per_epoch=resampled_steps_per_epoch,
#                     #callbacks=[early_stopping],
#                     validation_data=validation_data,
#                     class_weight={0:0.1, 1:0.1})

In [39]:
# resampled_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

In [45]:
tf.keras.backend.clear_session()